# Pandas TA ([pandas_ta](https://github.com/twopirllc/pandas-ta)) Strategies for Custom Technical Analysis

## Topics
- What is a Pandas TA Strategy?
    - Builtin Strategies: __AllStrategy__ and __CommonStrategy__
    - Creating Strategies
- Watchlist Class
    - Strategy Management and Execution
- Indicator Composition/Chaining for more Complex Strategies
    - Comprehensive Example: _MACD and RSI Momo with BBANDS and SMAs 50 & 200 and Cumulative Log Returns_

In [1]:
%matplotlib inline
import datetime as dt

import pandas as pd
import pandas_ta as ta
from alphaVantageAPI.alphavantage import AlphaVantage  # pip install alphaVantage-api

from watchlist import Watchlist
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# What is a Pandas TA Strategy?
A _Strategy_ is a simple way to name and group your favorite TA indicators. Technically, a _Strategy_ is a simple Data Class to contain list of indicators and their parameters. __Note__: _Strategy_ is experimental and subject to change. Pandas TA comes with two basic Strategies: __AllStrategy__ and __CommonStrategy__.

## Strategy Requirements:
- _name_: Some short memorable string.  _Note_: Case-insensitive "All" is reserved.
- _ta_: A list of dicts containing keyword arguments to identify the indicator and the indicator's arguments

## Optional Requirements:
- _description_: A more detailed description of what the Strategy tries to capture. Default: None
- _created_: At datetime string of when it was created. Default: Automatically generated.

### Things to note:
- A Strategy will __fail__ when consumed by Pandas TA if there is no {"kind": "indicator name"} attribute.

# Builtin Examples

### All

In [2]:
AllStrategy = ta.AllStrategy
print("name =", AllStrategy.name)
print("description =", AllStrategy.description)
print("created =", AllStrategy.created)
print("ta =", AllStrategy.ta)

name = All
description = All the indicators with their default settings. Pandas TA default.
created = 09/25/2020, 07:53:11
ta = None


### Common

In [3]:
CommonStrategy = ta.CommonStrategy
print("name =", CommonStrategy.name)
print("description =", CommonStrategy.description)
print("created =", CommonStrategy.created)
print("ta =", CommonStrategy.ta)

name = Common Price and Volume SMAs
description = Common Price SMAs: 10, 20, 50, 200 and Volume SMA: 20.
created = 09/25/2020, 07:53:11
ta = [{'kind': 'sma', 'length': 10}, {'kind': 'sma', 'length': 20}, {'kind': 'sma', 'length': 50}, {'kind': 'sma', 'length': 200}, {'kind': 'sma', 'close': 'volume', 'length': 20, 'prefix': 'VOL'}]


# Creating Strategies

### Simple Strategy A

In [4]:
custom_a = ta.Strategy(name="A", ta=[{"kind": "sma", "length": 50}, {"kind": "sma", "length": 200}])
custom_a

Strategy(name='A', ta=[{'kind': 'sma', 'length': 50}, {'kind': 'sma', 'length': 200}], description=None, created='09/25/2020, 07:53:11')

### Simple Strategy B

In [5]:
custom_b = ta.Strategy(name="B", ta=[{"kind": "ema", "length": 8}, {"kind": "ema", "length": 21}, {"kind": "log_return", "cumulative": True}, {"kind": "rsi"}, {"kind": "supertrend"}])
custom_b

Strategy(name='B', ta=[{'kind': 'ema', 'length': 8}, {'kind': 'ema', 'length': 21}, {'kind': 'log_return', 'cumulative': True}, {'kind': 'rsi'}, {'kind': 'supertrend'}], description=None, created='09/25/2020, 07:53:11')

### Bad Strategy. (Misspelled Indicator)

In [6]:
# Misspelled indicator, will fail later when ran with Pandas
custom_run_failure = ta.Strategy(name="Runtime Failure", ta=[{"kind": "percet_return"}])
custom_run_failure

Strategy(name='Runtime Failure', ta=[{'kind': 'percet_return'}], description=None, created='09/25/2020, 07:53:11')

# Strategy Management and Execution with _Watchlist_

### Initialize AlphaVantage Data Source

In [7]:
AV = AlphaVantage(
    api_key="YOUR API KEY", premium=False,
    output_size='full', clean=True,
    export_path=".", export=True
)
AV

AlphaVantage(
  end_point:str = https://www.alphavantage.co/query,
  api_key:str = YOUR API KEY,
  export:bool = True,
  export_path:str = .,
  output_size:str = full,
  output:str = csv,
  datatype:str = json,
  clean:bool = True,
  proxy:dict = {}
)

### Create Watchlist and set it's 'ds' to AlphaVantage

In [8]:
watch = Watchlist(["SPY", "IWM"], ds=AV)

#### Info about the Watchlist. Note, the default Strategy is "All"

In [9]:
watch

Watch(name='Watchlist: SPY, IWM', tickers[2]='SPY, IWM', tf='D', strategy[5]='Common Price and Volume SMAs')

### Help about Watchlist

In [10]:
help(Watchlist)

Help on class Watchlist in module watchlist:

class Watchlist(builtins.object)
 |  Watchlist(tickers: list, tf: str = None, name: str = None, strategy: pandas_ta.core.Strategy = None, ds: object = None, **kwargs)
 |  
 |  # Watchlist Class (** This is subject to change! **)
 |  A simple Class to load/download financial market data and automatically
 |  apply Technical Analysis indicators with a Pandas TA Strategy.
 |  
 |  Default Strategy: pandas_ta.AllStrategy.
 |  
 |  ## Package Support:
 |  ### Data Source (Default: AlphaVantage)
 |  - AlphaVantage (pip install alphaVantage-api).
 |  - Python Binance (pip install python-binance). # Future Support
 |  - Yahoo Finance (pip install yfinance). # Almost Supported
 |  
 |  # Technical Analysis:
 |  - Pandas TA (pip install pandas_ta)
 |  
 |  ## Required Arguments:
 |  - tickers: A list of strings containing tickers. Example: ['SPY', 'AAPL']
 |  
 |  Methods defined here:
 |  
 |  __init__(self, tickers: list, tf: str = None, name: str 

### Default Strategy is "Common"

In [11]:
# No arguments loads all the tickers and applies the Strategy to each ticker.
# The result can be accessed with Watchlist's 'data' property which returns a 
# dictionary keyed by ticker and DataFrames as values 
watch.load(verbose=True, timed=False)

[!] Loading All: SPY, IWM
[i] Loaded['D']: SPY_D.csv
                open      high       low     close      volume
date                                                          
1999-11-01  136.5000  137.0000  135.5625  135.5625   4006500.0
1999-11-02  135.9687  137.2500  134.5937  134.5937   6516900.0
1999-11-03  136.0000  136.3750  135.1250  135.5000   7222300.0
1999-11-04  136.7500  137.3593  135.7656  136.5312   7907500.0
1999-11-05  138.6250  139.1093  136.7812  137.8750   7431500.0
...              ...       ...       ...       ...         ...
2020-08-24  342.1200  343.0000  339.4504  342.9200  48588662.0
2020-08-25  343.5300  344.2100  342.2700  344.1200  38463381.0
2020-08-26  344.7600  347.8600  344.1700  347.5700  50790237.0
2020-08-27  348.5100  349.9000  346.5300  348.3300  58034142.0
2020-08-28  349.4400  350.7200  348.1500  350.5800  48588940.0

[5241 rows x 5 columns]
[+] Strategy: Common Price and Volume SMAs
[i] Indicator arguments: {'timed': False, 'append': True}
[i

In [12]:
watch.data

{'SPY':                 open      high       low     close      volume   SMA_10  \
 date                                                                      
 1999-11-01  136.5000  137.0000  135.5625  135.5625   4006500.0      NaN   
 1999-11-02  135.9687  137.2500  134.5937  134.5937   6516900.0      NaN   
 1999-11-03  136.0000  136.3750  135.1250  135.5000   7222300.0      NaN   
 1999-11-04  136.7500  137.3593  135.7656  136.5312   7907500.0      NaN   
 1999-11-05  138.6250  139.1093  136.7812  137.8750   7431500.0      NaN   
 ...              ...       ...       ...       ...         ...      ...   
 2020-08-24  342.1200  343.0000  339.4504  342.9200  48588662.0  337.837   
 2020-08-25  343.5300  344.2100  342.2700  344.1200  38463381.0  338.969   
 2020-08-26  344.7600  347.8600  344.1700  347.5700  50790237.0  339.982   
 2020-08-27  348.5100  349.9000  346.5300  348.3300  58034142.0  341.132   
 2020-08-28  349.4400  350.7200  348.1500  350.5800  48588940.0  342.506   
 
   

### 

In [13]:
watch.data["SPY"]

,open,high,low,close,volume,SMA_10,SMA_20,SMA_50,SMA_200,VOL_SMA_20
date,,,,,,,,,,
1999-11-01,136.5000,137.0000,135.5625,135.5625,4006500.0,NaN,NaN,NaN,NaN,NaN
1999-11-02,135.9687,137.2500,134.5937,134.5937,6516900.0,NaN,NaN,NaN,NaN,NaN
1999-11-03,136.0000,136.3750,135.1250,135.5000,7222300.0,NaN,NaN,NaN,NaN,NaN
1999-11-04,136.7500,137.3593,135.7656,136.5312,7907500.0,NaN,NaN,NaN,NaN,NaN
1999-11-05,138.6250,139.1093,136.7812,137.8750,7431500.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-08-24,342.1200,343.0000,339.4504,342.9200,48588662.0,337.837,333.5285,322.2548,307.05540,51792231.45
2020-08-25,343.5300,344.2100,342.2700,344.1200,38463381.0,338.969,334.6760,322.9962,307.23510,50840651.55
2020-08-26,344.7600,347.8600,344.1700,347.5700,50790237.0,339.982,335.7985,323.6884,307.42825,50957455.45


## Easy to swap Strategies and run them

### Running Simple Strategy A

In [14]:
# Load custom_a into Watchlist and verify
watch.strategy = custom_a
# watch.debug = True
watch.strategy

Strategy(name='A', ta=[{'kind': 'sma', 'length': 50}, {'kind': 'sma', 'length': 200}], description=None, created='09/25/2020, 07:53:11')

In [15]:
watch.load("IWM")

[i] Loaded['D']: IWM_D.csv
              open    high      low   close      volume
date                                                   
2000-05-26   91.06   91.44   90.630   91.44     37400.0
2000-05-30   92.75   94.81   92.750   94.81     28800.0
2000-05-31   95.13   96.38   95.130   95.75     18000.0
2000-06-01   97.11   97.31   97.110   97.31      3500.0
2000-06-02  101.70  102.40  101.700  102.40     14700.0
...            ...     ...      ...     ...         ...
2020-08-31  157.19  157.37  155.300  155.43  17051511.0
2020-09-01  155.22  157.31  154.450  157.21  15654144.0
2020-09-02  157.96  158.98  156.175  158.46  16763449.0
2020-09-03  158.12  158.29  152.960  153.78  32117585.0
2020-09-04  155.71  155.89  149.290  152.80  30618783.0

[5102 rows x 5 columns]


,open,high,low,close,volume,SMA_50,SMA_200
date,,,,,,,
2000-05-26,91.06,91.44,90.630,91.44,37400.0,NaN,NaN
2000-05-30,92.75,94.81,92.750,94.81,28800.0,NaN,NaN
2000-05-31,95.13,96.38,95.130,95.75,18000.0,NaN,NaN
2000-06-01,97.11,97.31,97.110,97.31,3500.0,NaN,NaN
2000-06-02,101.70,102.40,101.700,102.40,14700.0,NaN,NaN
...,...,...,...,...,...,...,...
2020-08-31,157.19,157.37,155.300,155.43,17051511.0,148.8200,145.82245
2020-09-01,155.22,157.31,154.450,157.21,15654144.0,149.1162,145.81800
2020-09-02,157.96,158.98,156.175,158.46,16763449.0,149.4254,145.81570


### Running Simple Strategy B

In [16]:
# Load custom_b into Watchlist and verify
watch.strategy = custom_b
watch.strategy

Strategy(name='B', ta=[{'kind': 'ema', 'length': 8}, {'kind': 'ema', 'length': 21}, {'kind': 'log_return', 'cumulative': True}, {'kind': 'rsi'}, {'kind': 'supertrend'}], description=None, created='09/25/2020, 07:53:11')

In [17]:
watch.load("SPY")

[i] Loaded['D']: SPY_D.csv
                open      high       low     close      volume
date                                                          
1999-11-01  136.5000  137.0000  135.5625  135.5625   4006500.0
1999-11-02  135.9687  137.2500  134.5937  134.5937   6516900.0
1999-11-03  136.0000  136.3750  135.1250  135.5000   7222300.0
1999-11-04  136.7500  137.3593  135.7656  136.5312   7907500.0
1999-11-05  138.6250  139.1093  136.7812  137.8750   7431500.0
...              ...       ...       ...       ...         ...
2020-08-24  342.1200  343.0000  339.4504  342.9200  48588662.0
2020-08-25  343.5300  344.2100  342.2700  344.1200  38463381.0
2020-08-26  344.7600  347.8600  344.1700  347.5700  50790237.0
2020-08-27  348.5100  349.9000  346.5300  348.3300  58034142.0
2020-08-28  349.4400  350.7200  348.1500  350.5800  48588940.0

[5241 rows x 5 columns]


,open,high,low,close,volume,EMA_8,EMA_21,CUMLOGRET_1,RSI_14,SUPERT_7_3.0,SUPERTd_7_3.0,SUPERTl_7_3.0,SUPERTs_7_3.0
date,,,,,,,,,,,,,
1999-11-01,136.5000,137.0000,135.5625,135.5625,4006500.0,NaN,NaN,NaN,NaN,0.000000,1,NaN,NaN
1999-11-02,135.9687,137.2500,134.5937,134.5937,6516900.0,NaN,NaN,-0.007172,0.000000,131.968750,1,131.968750,NaN
1999-11-03,136.0000,136.3750,135.1250,135.5000,7222300.0,NaN,NaN,-0.000461,50.185503,131.968750,1,131.968750,NaN
1999-11-04,136.7500,137.3593,135.7656,136.5312,7907500.0,NaN,NaN,0.007120,69.153995,131.968750,1,131.968750,NaN
1999-11-05,138.6250,139.1093,136.7812,137.8750,7431500.0,NaN,NaN,0.016915,79.896816,131.968750,1,131.968750,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-24,342.1200,343.0000,339.4504,342.9200,48588662.0,338.577882,333.520811,0.928064,72.830131,331.974175,1,331.974175,NaN
2020-08-25,343.5300,344.2100,342.2700,344.1200,38463381.0,339.809464,334.484374,0.931558,74.054445,334.479122,1,334.479122,NaN
2020-08-26,344.7600,347.8600,344.1700,347.5700,50790237.0,341.534027,335.673976,0.941533,77.231098,336.902819,1,336.902819,NaN


### Running Bad Strategy. (Misspelled indicator)

In [18]:
# Load custom_run_failure into Watchlist and verify
watch.strategy = custom_run_failure
watch.strategy

Strategy(name='Runtime Failure', ta=[{'kind': 'percet_return'}], description=None, created='09/25/2020, 07:53:11')

In [19]:
try:
    iwm = watch.load("IWM")
except AttributeError as error:
    print(f"[X] Oops! {error}")

[i] Loaded['D']: IWM_D.csv
              open    high      low   close      volume
date                                                   
2000-05-26   91.06   91.44   90.630   91.44     37400.0
2000-05-30   92.75   94.81   92.750   94.81     28800.0
2000-05-31   95.13   96.38   95.130   95.75     18000.0
2000-06-01   97.11   97.31   97.110   97.31      3500.0
2000-06-02  101.70  102.40  101.700  102.40     14700.0
...            ...     ...      ...     ...         ...
2020-08-31  157.19  157.37  155.300  155.43  17051511.0
2020-09-01  155.22  157.31  154.450  157.21  15654144.0
2020-09-02  157.96  158.98  156.175  158.46  16763449.0
2020-09-03  158.12  158.29  152.960  153.78  32117585.0
2020-09-04  155.71  155.89  149.290  152.80  30618783.0

[5102 rows x 5 columns]
[X] Oops! 'AnalysisIndicators' object has no attribute 'percet_return'


# Indicator Composition/Chaining
- When you need an indicator to depend on the value of a prior indicator
- Utilitze _prefix_ or _suffix_ to help identify unique columns or avoid column name clashes.

### Volume MAs and MA chains

In [20]:
# Set EMA's and SMA's 'close' to 'volume' to create Volume MAs, prefix 'volume' MAs with 'VOLUME' so easy to identify the column
# Take a price EMA and apply LINREG from EMA's output
volmas_price_ma_chain = [
    {"kind":"ema", "close": "volume", "length": 10, "prefix": "VOLUME"},
    {"kind":"sma", "close": "volume", "length": 20, "prefix": "VOLUME"},
    {"kind":"ema", "length": 5},
    {"kind":"linreg", "close": "EMA_5", "length": 8, "prefix": "EMA_5"},
]
vp_ma_chain_ta = ta.Strategy("Volume MAs and Price MA chain", volmas_price_ma_chain)
vp_ma_chain_ta

Strategy(name='Volume MAs and Price MA chain', ta=[{'kind': 'ema', 'close': 'volume', 'length': 10, 'prefix': 'VOLUME'}, {'kind': 'sma', 'close': 'volume', 'length': 20, 'prefix': 'VOLUME'}, {'kind': 'ema', 'length': 5}, {'kind': 'linreg', 'close': 'EMA_5', 'length': 8, 'prefix': 'EMA_5'}], description=None, created='09/25/2020, 07:53:11')

In [21]:
# Update the Watchlist
watch.strategy = vp_ma_chain_ta
watch.strategy.name

'Volume MAs and Price MA chain'

In [22]:
spy = watch.load("SPY")
spy

[i] Loaded['D']: SPY_D.csv
                open      high       low     close      volume
date                                                          
1999-11-01  136.5000  137.0000  135.5625  135.5625   4006500.0
1999-11-02  135.9687  137.2500  134.5937  134.5937   6516900.0
1999-11-03  136.0000  136.3750  135.1250  135.5000   7222300.0
1999-11-04  136.7500  137.3593  135.7656  136.5312   7907500.0
1999-11-05  138.6250  139.1093  136.7812  137.8750   7431500.0
...              ...       ...       ...       ...         ...
2020-08-24  342.1200  343.0000  339.4504  342.9200  48588662.0
2020-08-25  343.5300  344.2100  342.2700  344.1200  38463381.0
2020-08-26  344.7600  347.8600  344.1700  347.5700  50790237.0
2020-08-27  348.5100  349.9000  346.5300  348.3300  58034142.0
2020-08-28  349.4400  350.7200  348.1500  350.5800  48588940.0

[5241 rows x 5 columns]


,open,high,low,close,volume,VOLUME_EMA_10,VOLUME_SMA_20,EMA_5,EMA_5_LR_8
date,,,,,,,,,
1999-11-01,136.5000,137.0000,135.5625,135.5625,4006500.0,NaN,NaN,NaN,NaN
1999-11-02,135.9687,137.2500,134.5937,134.5937,6516900.0,NaN,NaN,NaN,NaN
1999-11-03,136.0000,136.3750,135.1250,135.5000,7222300.0,NaN,NaN,NaN,NaN
1999-11-04,136.7500,137.3593,135.7656,136.5312,7907500.0,NaN,NaN,NaN,NaN
1999-11-05,138.6250,139.1093,136.7812,137.8750,7431500.0,NaN,NaN,136.012480,NaN
...,...,...,...,...,...,...,...,...,...
2020-08-24,342.1200,343.0000,339.4504,342.9200,48588662.0,4.999906e+07,51792231.45,339.779956,338.589691
2020-08-25,343.5300,344.2100,342.2700,344.1200,38463381.0,4.790167e+07,50840651.55,341.226637,339.658445
2020-08-26,344.7600,347.8600,344.1700,347.5700,50790237.0,4.842686e+07,50957455.45,343.341091,341.152455


### MACD BBANDS

In [23]:
# MACD is the initial indicator that BBANDS depends on.
# Set BBANDS's 'close' to MACD's main signal, in this case 'MACD_12_26_9' and add a prefix (or suffix) so it's easier to identify
macd_bands_ta = [
    {"kind":"macd"},
    {"kind":"bbands", "close": "MACD_12_26_9", "length": 20, "prefix": "MACD"}
]
macd_bands_ta = ta.Strategy("MACD BBands", macd_bands_ta, f"BBANDS_{macd_bands_ta[1]['length']} applied to MACD")
macd_bands_ta

Strategy(name='MACD BBands', ta=[{'kind': 'macd'}, {'kind': 'bbands', 'close': 'MACD_12_26_9', 'length': 20, 'prefix': 'MACD'}], description='BBANDS_20 applied to MACD', created='09/25/2020, 07:53:11')

In [24]:
# Update the Watchlist
watch.strategy = macd_bands_ta
watch.strategy.name

'MACD BBands'

In [25]:
spy = watch.load("SPY")
spy

[i] Loaded['D']: SPY_D.csv
                open      high       low     close      volume
date                                                          
1999-11-01  136.5000  137.0000  135.5625  135.5625   4006500.0
1999-11-02  135.9687  137.2500  134.5937  134.5937   6516900.0
1999-11-03  136.0000  136.3750  135.1250  135.5000   7222300.0
1999-11-04  136.7500  137.3593  135.7656  136.5312   7907500.0
1999-11-05  138.6250  139.1093  136.7812  137.8750   7431500.0
...              ...       ...       ...       ...         ...
2020-08-24  342.1200  343.0000  339.4504  342.9200  48588662.0
2020-08-25  343.5300  344.2100  342.2700  344.1200  38463381.0
2020-08-26  344.7600  347.8600  344.1700  347.5700  50790237.0
2020-08-27  348.5100  349.9000  346.5300  348.3300  58034142.0
2020-08-28  349.4400  350.7200  348.1500  350.5800  48588940.0

[5241 rows x 5 columns]


,open,high,low,close,volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,MACD_BBL_20_2.0,MACD_BBM_20_2.0,MACD_BBU_20_2.0
date,,,,,,,,,,,
1999-11-01,136.5000,137.0000,135.5625,135.5625,4006500.0,NaN,NaN,NaN,NaN,NaN,NaN
1999-11-02,135.9687,137.2500,134.5937,134.5937,6516900.0,NaN,NaN,NaN,NaN,NaN,NaN
1999-11-03,136.0000,136.3750,135.1250,135.5000,7222300.0,NaN,NaN,NaN,NaN,NaN,NaN
1999-11-04,136.7500,137.3593,135.7656,136.5312,7907500.0,NaN,NaN,NaN,NaN,NaN,NaN
1999-11-05,138.6250,139.1093,136.7812,137.8750,7431500.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2020-08-24,342.1200,343.0000,339.4504,342.9200,48588662.0,5.341674,0.069881,5.271794,3.678022,4.914752,6.151482
2020-08-25,343.5300,344.2100,342.2700,344.1200,38463381.0,5.513019,0.192980,5.320039,3.807707,4.990945,6.174183
2020-08-26,344.7600,347.8600,344.1700,347.5700,50790237.0,5.859651,0.431690,5.427961,3.933776,5.083307,6.232838


# Comprehensive Strategy

### MACD and RSI Momentum with BBANDS and SMAs and Cumulative Log Returns

In [26]:
momo_bands_sma_ta = [
    {"kind":"sma", "length": 50},
    {"kind":"sma", "length": 200},
    {"kind":"bbands", "length": 20},
    {"kind":"macd"},
    {"kind":"rsi"},
    {"kind":"log_return", "cumulative": True},
    {"kind":"sma", "close": "CUMLOGRET_1", "length": 5, "suffix": "CUMLOGRET"},
]
momo_bands_sma_strategy = ta.Strategy(
    "Momo, Bands and SMAs and Cumulative Log Returns", # name
    momo_bands_sma_ta, # ta
    "MACD and RSI Momo with BBANDS and SMAs 50 & 200 and Cumulative Log Returns" # description
)
momo_bands_sma_strategy

Strategy(name='Momo, Bands and SMAs and Cumulative Log Returns', ta=[{'kind': 'sma', 'length': 50}, {'kind': 'sma', 'length': 200}, {'kind': 'bbands', 'length': 20}, {'kind': 'macd'}, {'kind': 'rsi'}, {'kind': 'log_return', 'cumulative': True}, {'kind': 'sma', 'close': 'CUMLOGRET_1', 'length': 5, 'suffix': 'CUMLOGRET'}], description='MACD and RSI Momo with BBANDS and SMAs 50 & 200 and Cumulative Log Returns', created='09/25/2020, 07:53:11')

In [27]:
# Update the Watchlist
watch.strategy = momo_bands_sma_strategy
watch.strategy.name

'Momo, Bands and SMAs and Cumulative Log Returns'

In [28]:
spy = watch.load("SPY", timed=True)
# Apply constants to the DataFrame for indicators
spy.ta.constants(True, [0, 30, 70])
spy.head()

[i] Loaded['D']: SPY_D.csv
                open      high       low     close      volume
date                                                          
1999-11-01  136.5000  137.0000  135.5625  135.5625   4006500.0
1999-11-02  135.9687  137.2500  134.5937  134.5937   6516900.0
1999-11-03  136.0000  136.3750  135.1250  135.5000   7222300.0
1999-11-04  136.7500  137.3593  135.7656  136.5312   7907500.0
1999-11-05  138.6250  139.1093  136.7812  137.8750   7431500.0
...              ...       ...       ...       ...         ...
2020-08-24  342.1200  343.0000  339.4504  342.9200  48588662.0
2020-08-25  343.5300  344.2100  342.2700  344.1200  38463381.0
2020-08-26  344.7600  347.8600  344.1700  347.5700  50790237.0
2020-08-27  348.5100  349.9000  346.5300  348.3300  58034142.0
2020-08-28  349.4400  350.7200  348.1500  350.5800  48588940.0

[5241 rows x 5 columns]
[i] Runtime: 927.2966 ms (0.9273 s)


,open,high,low,close,volume,SMA_50,SMA_200,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,CUMLOGRET_1,SMA_5_CUMLOGRET,0,30,70
date,,,,,,,,,,,,,,,,,,,
1999-11-01,136.5000,137.0000,135.5625,135.5625,4006500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,30,70
1999-11-02,135.9687,137.2500,134.5937,134.5937,6516900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.007172,NaN,0,30,70
1999-11-03,136.0000,136.3750,135.1250,135.5000,7222300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.185503,-0.000461,NaN,0,30,70
1999-11-04,136.7500,137.3593,135.7656,136.5312,7907500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.153995,0.007120,NaN,0,30,70
1999-11-05,138.6250,139.1093,136.7812,137.8750,7431500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.896816,0.016915,NaN,0,30,70


# Additional Strategy Options

The ```params``` keyword takes a _tuple_ as a shorthand to the parameter arguments in order.
* **Note**: If the indicator arguments change, so will results. Breaking Changes will **always** be posted on the README.

The ```col_numbers``` keyword takes a _tuple_ specifying which column to return if the result is a DataFrame.

In [29]:
params_ta = [
    {"kind":"ema", "params": (10,)},
    # params sets MACD's keyword arguments: fast=9, slow=19, signal=10
    # and returning the 2nd column: histogram
    {"kind":"macd", "params": (9, 19, 10), "col_numbers": (1,)},
    # Selects the Lower and Upper Bands and renames them LB and UB, ignoring the MB
    {"kind":"bbands", "col_numbers": (0,2), "col_names": ("LB", "UB")},
    {"kind":"log_return", "params": (5, False)},
]
params_ta_strategy = ta.Strategy(
    "EMA, MACD History, Outter BBands, Log Returns", # name
    params_ta, # ta
    "EMA, MACD History, BBands(LB, UB), and Log Returns Strategy" # description
)
params_ta_strategy

Strategy(name='EMA, MACD History, Outter BBands, Log Returns', ta=[{'kind': 'ema', 'params': (10,)}, {'kind': 'macd', 'params': (9, 19, 10), 'col_numbers': (1,)}, {'kind': 'bbands', 'col_numbers': (0, 2), 'col_names': ('LB', 'UB')}, {'kind': 'log_return', 'params': (5, False)}], description='EMA, MACD History, BBands(LB, UB), and Log Returns Strategy', created='09/25/2020, 07:53:11')

In [30]:
# Update the Watchlist
watch.strategy = params_ta_strategy
watch.strategy.name

'EMA, MACD History, Outter BBands, Log Returns'

In [31]:
spy = watch.load("SPY", timed=True)
spy.tail()

[i] Loaded['D']: SPY_D.csv
                open      high       low     close      volume
date                                                          
1999-11-01  136.5000  137.0000  135.5625  135.5625   4006500.0
1999-11-02  135.9687  137.2500  134.5937  134.5937   6516900.0
1999-11-03  136.0000  136.3750  135.1250  135.5000   7222300.0
1999-11-04  136.7500  137.3593  135.7656  136.5312   7907500.0
1999-11-05  138.6250  139.1093  136.7812  137.8750   7431500.0
...              ...       ...       ...       ...         ...
2020-08-24  342.1200  343.0000  339.4504  342.9200  48588662.0
2020-08-25  343.5300  344.2100  342.2700  344.1200  38463381.0
2020-08-26  344.7600  347.8600  344.1700  347.5700  50790237.0
2020-08-27  348.5100  349.9000  346.5300  348.3300  58034142.0
2020-08-28  349.4400  350.7200  348.1500  350.5800  48588940.0

[5241 rows x 5 columns]
[i] Runtime: 180.5198 ms (0.1805 s)


,open,high,low,close,volume,EMA_10,MACDh_9_19_10,LB,UB,LOGRET_5
date,,,,,,,,,,
2020-08-24,342.12,343.00,339.4504,342.92,48588662.0,337.797677,-0.000276,334.962909,343.657091,0.014718
2020-08-25,343.53,344.21,342.2700,344.12,38463381.0,338.947190,0.165373,334.441244,346.370756,0.016053
2020-08-26,344.76,347.86,344.1700,347.57,50790237.0,340.514974,0.469555,335.028792,349.919208,0.030201
2020-08-27,348.51,349.90,346.5300,348.33,58034142.0,341.935888,0.613083,337.277495,351.690505,0.029276
2020-08-28,349.44,350.72,348.1500,350.58,48588940.0,343.507545,0.771996,340.426029,352.981971,0.032174
